## DC2 Retrieve Forced Src catalogs of visit-level forced photometry from coadd Object catalog.  
Michael Wood-Vasey

This Notebook shows how to retrieve the list of forced sources from a given tract, patch, filter.

The forced-source dataIds contain sufficient information to look up the calexp IDs to retrieve postage stamps.

In [ ]:
import numpy as np

import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage

In [ ]:
repo = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/output'
butler = dafPersist.Butler(repo)

In [ ]:
tract = 4850
patch = '5,6'

filt = 'r'
partial_data_id = {'tract': tract, 'patch': patch, 'filter': filt}

In [ ]:
dataset_type = 'forced_src'
data_refs = butler.subset(datasetType=dataset_type, dataId=partial_data_id)
data_ids = [dr.dataId for dr in data_refs
           if butler.datasetExists(datasetType=dataset_type,
                                   dataId=dr.dataId)]

In [ ]:
print(data_ids[0:3])

In [ ]:
dId = {'tract': 4848, 'patch': '5,6', 'filter': 'r', 'visit': 193861, 'raft': '1,4', 'sensor': '0,0'}

calibrated_exposure = butler.get(datasetType='calexp', dataId=dId)
forced_src = butler.get(datasetType=dataset_type, dataId=dId)

In [ ]:
cat = forced_src.asAstropy()

In [ ]:
calib = calibrated_exposure.getCalib()
calib.setThrowOnNegativeFlux(False)

mag, mag_err = calib.getMagnitude(forced_src['base_PsfFlux_flux'],
                                  forced_src['base_PsfFlux_fluxSigma'])
cat['mag'] = mag
cat['mag_err'] = mag_err
cat['snr'] = np.abs(cat['base_PsfFlux_flux'])/cat['base_PsfFlux_fluxSigma']

In [ ]:
print(cat)

This is per tract, so 22,062 sources seems potentially reasonble.

The `id` is the same `id` as in the coadd "reference" catalog (the result of merged detections from the individ

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
plt.scatter(np.rad2deg(cat['coord_ra']),
            np.rad2deg(cat['coord_dec']))
plt.xlabel('RA [deg]')
plt.ylabel('Dec [deg]')

In [ ]:
plt.scatter(cat['mag'], cat['snr'])
plt.xlabel('%s mag' % dId['filter'])
plt.ylabel('S/N')

Looks like a reasonable set of magnitudes and uncertainties.

## From one image to many
Let's pick one of the Objects and go through all of the data IDs in that filter for the tract, patch.

In [ ]:
object_id = cat[0]['id']

In [ ]:
def get_all_images_for_object_id_from_data_ids(object_id, data_refs, dataset_type='forced_src'):
    """Take an Object ID and data_ref list and return matching data_refs that contain that object and exist
    
    Note:  This perhaps isn't quite the function you wanted.
    You may have wanted a function that just took an object_id and filter.
    But we'll get there.
    
    Note that we're being a little inefficient and loading each raft, sensor catalog
    and checking to see if it contains the given ID.
    It would likely be more efficient to just load the WCS for the Visit
    and look up the raft, sensor.
    """
    matching_data_refs = []
    for data_ref in data_refs:
        # Should add check to make sure it exists
        if not data_ref.datasetExists(datasetType=dataset_type):
            continue
        cat = data_ref.get(datasetType=dataset_type)
        if object_id in cat['id']:
            matching_data_refs.append(data_ref)

    return matching_data_refs

In [ ]:
matching_data_refs = get_all_images_for_object_id_from_data_ids(object_id, data_refs, dataset_type='forced_src')

We're still missing one step.

How do we get the tract, patch for a given ObjectId or RA, Dec?

You probably wanted to start with a given ObjectID from the Object (coadd) catalog and then get all of the images that include that object.

To do this we
1. Get the skymap for the coadd dataset (by default, `deepCoadd`)
2. Use the skymap object to look up the tract
3. Use the tract object to look up the patch
4. Create a partial data Id dict and query the butler for `forced_src` catalogs that match this partial data Id.
5. Go through each forced_src catalog in that tract, patch and save the ones that match the given Id.

In [ ]:
skymap = butler.get(datasetType='deepCoadd_skyMap')

In [ ]:
help(skymap.findTract)

In [ ]:
obj = cat[100]
ra, dec = obj['coord_ra'], obj['coord_dec']
# Note the catalog returns coord_ra, coord_dec in RADIANS
radec = afwGeom.SpherePoint(ra, dec, afwGeom.radians)

tractInfo = skymap.findTract(radec)
patchInfo = tractInfo.findPatch(radec)

In [ ]:
filt = 'r'
partial_data_id = {'tract': tractInfo.getId(),
                   'patch': "%d,%d" % patchInfo.getIndex(),
                   'filter': filt}

dataset_type = 'forced_src'
data_refs = butler.subset(datasetType=dataset_type, dataId=partial_data_id)
data_ids = [dr.dataId for dr in data_refs
           if butler.datasetExists(datasetType=dataset_type,
                                   dataId=dr.dataId)]

In [ ]:
object = obj['id']
matching_data_refs = get_all_images_for_object_id_from_data_ids(object_id, data_refs, dataset_type='forced_src')

In [ ]:
data_ids = [dr.dataId for dr in matching_data_refs]

We could now use these data_ids to generate postage stamps from the calexps

In [ ]:
def cutout_ra_dec(butler, ra, dec, data_id, datasetType='calexp',
                  cutoutSideLength=51, verbose=False,
                  **kwargs):
    """
    Produce a cutout from the given image at the given afw SpherePoint radec position.
    
    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra, dec: Right Ascension, Declination in decimal degrees
        Coordinates of the center of the cutout.
    data_id: Data Id
    datasetType: string ['calexp']  
    cutoutSideLength: float [optional] 
        Side of the cutout region in pixels.
    
    Returns
    -------
    MaskedImage
    """
    cutoutSize = afwGeom.ExtentI(cutoutSideLength, cutoutSideLength)

    radec = afwGeom.SpherePoint(ra, dec, afwGeom.degrees)

    calexp = butler.get(datasetType, dataId=data_id)
    xy = afwGeom.PointI(calexp.getWcs().skyToPixel(radec))
    if verbose:
        print("Making cutout at (x, y) {xy:} of size ({cutoutSize:}, {cutoutSize:})".format({'xy': xy, 'cutoutSize': cutoutSize}))
        print(xy, cutoutSize)

    bbox = afwGeom.BoxI(xy - cutoutSize//2, cutoutSize)
    
    cutout_image = butler.get(datasetType+'_sub', bbox=bbox, immediate=True, dataId=data_id)
    
    return cutout_image

In [ ]:
def display_cutout_image(butler, ra, dec, data_id,
                         vmin=None, vmax=None, label=None,
                         frame=None, display=None, backend='matplotlib',
                         show=True, saveplot=False, savefits=False,
                         datasetType='calexp'):
    """
    Display a postage stamp for a given RA, Dec using LSST lsst.afw.display.
    
    Parameters
    ----------
    ra: float [degrees]
    dec: float [degrees]
    backend: string
        Backend can be anything that lsst.afw.display and your configuration supports: 
        e.g. matplotlib, ds9, ginga, firefly.
    
    Returns
    -------
    MaskedImage
    
    Notes
    -----
    Parameters are the same as for make_cutout_image, except for the backend.
    You definitely have the matplotlib backend.
    ds9, ginga, and firefly can be set up but are non-trivial on the scale of a simple Notebook.
    """
    cutout_image = cutout_ra_dec(butler, ra, dec, data_id, datasetType='deepCoadd')
    if savefits:
        if isinstance(savefits, str):
            filename = savefits
        else:
            filename = 'postage-stamp.fits'
        cutout_image.writeFits(filename)
    
    if display is None:
        display = afwDisplay.Display(frame=frame, backend=backend)

    radec = afwGeom.SpherePoint(ra, dec, afwGeom.degrees)
    xy = cutout_image.getWcs().skyToPixel(radec)
    
    display.mtv(cutout_image)
    display.scale("asinh", "zscale")
    display.dot('o', xy.getX(), xy.getY(), ctype='red')
    display.show_colorbar()
    
    return cutout_image

In [ ]:
ra_deg, dec_deg = np.rad2deg(ra), np.rad2deg(dec)

for did in data_ids:
    display_cutout_image(butler, ra_deg, dec_deg, did, datasetType='calexp')